In [ ]:
0)

# You can make it look nice
print("Mean:", round(mean_age, 2))
print("Median:", round(median_age, 2))
print("Mode:", round(mode_age, 2))
print("Standard Deviation:", round(std_dev_age, 2))
print("Variance:", round(variance_age, 2))
print("Max:", round(max_age, 2))
print("Min:", round(min_age, 2))
print("Range:", round(age_range, 2))
print("Deciles")
print(deciles_age)



1)

def visualize_null(dataframe,percentage=True):
    null_series=survey_df.isna().sum().sort_values(ascending=False)
    if percentage:
        null_series=null_series/len(dataframe)*100

    plt.bar(x=null_series.index,height=null_series.values)

    if percentage:
        plt.ylabel('Percentage of missing values')
    else:
        plt.ylabel('Amount of missing values')
    plt.title('Missing values')
    plt.xlabel('Columns')
    plt.xticks(rotation=90)
    plt.show()

visualize_null(survey_df)



2)

# All this exercise could have been done in just one very efficient step

data_path = "./data/"
survey_filename = "survey_music_noise.csv"
survey_df_data = pd.read_csv(data_path + survey_filename)
survey_df = survey_df_data.copy()

def survey_changes(survey_df):
    replace_dict_numeric={'Null':np.nan,'fifteen':15,'twenty':20}
    replace_dict_cat={'Null':np.nan,'0':np.nan,'-500':np.nan}


    survey_df['Timestamp'][survey_df['Timestamp'].str.len()<16]=np.nan

    survey_df['Timestamp']=survey_df['Timestamp'].str.replace('.',':')
    survey_df['Timestamp']=survey_df['Timestamp'].str.replace('-','/')


    survey_df['Timestamp']=pd.to_datetime(survey_df['Timestamp'])


    numeric_cols=['Age','BPM','Hours per day']
    for col in numeric_cols:
        survey_df[col]=survey_df[col].replace(replace_dict_numeric)

        # Now we can convert it to the correct dtype
        survey_df[col]=survey_df[col].astype(float)


    for col in ['Anxiety','Depression','Insomnia','OCD']:
        survey_df[col]=survey_df[col].replace(replace_dict_numeric)

        # Now we can convert it to the correct dtype
        survey_df[col]=survey_df[col].astype(float)


    cat_cols=['Primary streaming service','While working','Instrumentalist','Composer','Exploratory','Foreign languages','Fav genre','Music effects','Permissions']


    for col in cat_cols:
        survey_df[col]=survey_df[col].replace(replace_dict_cat)
        survey_df[col]=survey_df[col].str.strip()
        survey_df[col]=survey_df[col].str.lower()

        survey_df[col]=survey_df[col].astype('category')

    freq_mask=survey_df.columns.str.startswith('Freq')
    freq_columns=survey_df.columns[freq_mask]

    survey_df.loc[:,freq_columns]=survey_df.loc[:,freq_columns].replace(replace_dict_cat)
    for col in freq_columns:

        survey_df[col]=survey_df[col].str.lower()
        survey_df[col]=survey_df[col].str.strip()


    dict_replace={'never':0,'rarely':1,'sometimes':2,'very frequently':3}

    freq_mask=survey_df.columns.str.startswith('Freq')
    freq_columns=survey_df.columns[freq_mask]


    survey_df.loc[:,freq_columns]=survey_df.loc[:,freq_columns].replace(dict_replace)
    for col in freq_columns:
        survey_df[col]=survey_df[col].astype(float)

    return survey_df


survey_df=survey_changes(survey_df)



3)

def visualize_null(dataframe,percentage=True):
    null_series=survey_df.isna().sum().sort_values(ascending=False)
    if percentage:
        null_series=null_series/len(dataframe)*100

    plt.bar(x=null_series.index,height=null_series.values)

    if percentage:
        plt.ylabel('Percentage of missing values')
    else:
        plt.ylabel('Amount of missing values')
    plt.title('Missing values')
    plt.xlabel('Columns')
    plt.xticks(rotation=90)
    plt.show()

visualize_null(survey_df)



4)

# Import libraries
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

np.random.seed(42)



5)

corr_survey=survey_df.select_dtypes(np.number).corr()
levels_corr=np.zeros(shape=corr_survey.shape)

levels=np.arange(-3,4)
ranges=[(-1,-0.7),(-0.7,-0.4),(-0.4,-0.1),(-0.1,0.1),(0.1,0.3),(0.3,0.7),(0.7,1)]

for i,r in enumerate(ranges):
    lower,upper=r
    levels_corr[(corr_survey>=lower) & (corr_survey<upper)] =levels[i]

levels_corr=pd.DataFrame(data=levels_corr,index=corr_survey.index,columns=corr_survey.columns)


plt.figure(figsize=(15,15))
sns.heatmap(levels_corr, annot=True, cmap='coolwarm')



6)

# Selecting two genres for the T-Test: Classical and Rock
classical_hours = survey_df[survey_df['Fav genre'] == 'classical']['Hours per day']
rock_hours = survey_df[survey_df['Fav genre'] == 'rock']['Hours per day']

# Performing a T-Test
t_test_result = stats.ttest_ind(classical_hours, rock_hours, nan_policy='omit')

# Visualization: Box plot for Classical and Rock genres
plt.figure(figsize=(10, 6))
data_obj=survey_df[survey_df['Fav genre'].isin(['classical', 'rock'])]
data_obj['Fav genre']=data_obj['Fav genre'].astype(object) # We will make this change only to show a better visualizations. Otherwise we would get all categories in the boxplot
sns.boxplot(x='Fav genre', y='Hours per day', data=data_obj)
plt.title('Box Plot of Listening Hours for Classical and Rock Genres')
plt.show()

# T-Test Result
print(f'The p-value of the statistical test is: \n{t_test_result[1]:3.3f}')



7)

# Preparing data for the Chi-Square Test
contingency_table = pd.crosstab(survey_df['Fav genre'], survey_df['Music effects'])

# Performing Chi-Square Test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Visualization: Heatmap of the contingency table
plt.figure(figsize=(12, 8))
sns.heatmap(contingency_table, annot=True, cmap="YlGnBu", fmt='g')
plt.title('Heatmap of Fav Genre vs. Music Effects')
plt.ylabel('Favorite Genre')
plt.xlabel('Music Effects')
plt.show()

# Chi-Square Test Result
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)



8)

# Preparing data for Correlation Analysis
# Filtering out rows with missing values in 'Age' or 'Hours per day'
cleaned_data = survey_df[['Age', 'Hours per day']].dropna()

# Performing Correlation Analysis
correlation_result = cleaned_data.corr(method='pearson')

# Visualization: Scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(x='Age', y='Hours per day', data=cleaned_data, scatter_kws={'alpha':0.5})
plt.title('Scatter Plot with Regression Line: Age vs. Listening Hours')
plt.xlabel('Age')
plt.ylabel('Hours per Day Listening to Music')
plt.show()

# Correlation Result
correlation_result



9)

# Regression without train-test split

# Regression with BPM Only
X_bpm_only = survey_df[['BPM']].dropna()
y_bpm_only = survey_df['Hours per day'].dropna()
X_bpm_only, y_bpm_only = X_bpm_only.align(y_bpm_only, axis=0, join='inner')

# Regression with Age Only
X_age_only = survey_df[['Age']].dropna()
y_age_only = survey_df['Hours per day'].dropna()
X_age_only, y_age_only = X_age_only.align(y_age_only, axis=0, join='inner')


# Creating linear regression models for BPM and Age
model_bpm_no_split = LinearRegression()
model_age_no_split = LinearRegression()

# Fitting the models
model_bpm_no_split.fit(X_bpm_only, y_bpm_only)
model_age_no_split.fit(X_age_only, y_age_only)

# Predicting 'Hours per day' for the fitted data
y_bpm_pred_no_split = model_bpm_no_split.predict(X_bpm_only)
y_age_pred_no_split = model_age_no_split.predict(X_age_only)

# 2D Plots for BPM and Age regressions
plt.figure(figsize=(12, 6))

# BPM Plot
plt.subplot(1, 2, 1)
plt.scatter(X_bpm_only, y_bpm_only, alpha=0.5)
plt.plot(X_bpm_only, y_bpm_pred_no_split, color='red')
plt.xlabel('BPM')
plt.ylabel('Hours per Day')
plt.title('BPM vs. Hours per Day (No Split)')

# Age Plot
plt.subplot(1, 2, 2)
plt.scatter(X_age_only, y_age_only, alpha=0.5)
plt.plot(X_age_only, y_age_pred_no_split, color='red')
plt.xlabel('Age')
plt.ylabel('Hours per Day')
plt.title('Age vs. Hours per Day (No Split)')

plt.tight_layout()
plt.show()



10)

# Regression with train-test split

# Splitting the dataset for BPM regression
X_bpm_train_split, X_bpm_test_split, y_bpm_train_split, y_bpm_test_split = train_test_split(X_bpm_only, y_bpm_only, test_size=0.3, random_state=42)

# Splitting the dataset for Age regression
X_age_train_split, X_age_test_split, y_age_train_split, y_age_test_split = train_test_split(X_age_only, y_age_only, test_size=0.3, random_state=42)

# Creating and training the BPM and Age regression models
model_bpm_split = LinearRegression()
model_age_split = LinearRegression()
model_bpm_split.fit(X_bpm_train_split, y_bpm_train_split)
model_age_split.fit(X_age_train_split, y_age_train_split)

# Predicting 'Hours per day' for the test data
y_bpm_pred_split = model_bpm_split.predict(X_bpm_test_split)
y_age_pred_split = model_age_split.predict(X_age_test_split)

# 2D Plots for BPM and Age regressions with train-test split
plt.figure(figsize=(12, 6))

# BPM Plot
plt.subplot(1, 2, 1)
plt.scatter(X_bpm_train_split, y_bpm_train_split,color='blue', alpha=0.5, label='Train Data')
plt.scatter(X_bpm_test_split, y_bpm_test_split, alpha=0.5, label='Test Data')
plt.plot(X_bpm_test_split, y_bpm_pred_split, 'r-', label='Regression Line',color='green')
plt.xlabel('BPM')
plt.ylabel('Hours per Day')
plt.title('BPM vs. Hours per Day (With Split)')
plt.legend()

# Age Plot
plt.subplot(1, 2, 2)
plt.scatter(X_age_train_split, y_age_train_split,color='blue', alpha=0.5, label='Train Data')
plt.scatter(X_age_test_split, y_age_test_split, alpha=0.5, label='Test Data')
plt.plot(X_age_test_split, y_age_pred_split, 'r-', label='Regression Line',color='green')
plt.xlabel('Age')
plt.ylabel('Hours per Day')
plt.title('Age vs. Hours per Day (With Split)')
plt.legend()

plt.tight_layout()
plt.show()



11)

# Copy from Syllabus

music_data_1=survey_df.copy()

# Let's predict whether a person is an instrumentalist based on Age, Hours per day, and Fav genre
music_data_1['Instrumentalist'] = survey_df['Instrumentalist'].map({'yes': 1, 'no': 0})  # Convert to binary
music_data_1 = pd.get_dummies(survey_df, columns=['Fav genre'])  # One-hot encoding for 'Fav genre'

# Selecting predictor variables and target variable
X = music_data_1[['Age', 'Hours per day'] + [col for col in music_data_1 if col.startswith('Fav genre')]].dropna()
y = music_data_1['Instrumentalist'].dropna()

# Ensuring the lengths of X and y are the same after dropping NaN values
X, y = X.align(y, axis=0, join='inner')

# Splitting the music_dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating and training the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predicting the target variable for the test set
y_pred = model.predict(X_test)

# Model Evaluation
accuracy = model.score(X_test, y_test)
classification_report_result = classification_report(y_test, y_pred)
confusion_matrix_result = confusion_matrix(y_test, y_pred)

print(accuracy)
print(classification_report_result)



12)

# Creating an annotated heatmap from the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt="d", cmap='Blues', cbar=False)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix Heatmap')
plt.show()